# Module 1 - Programming Assignment


# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
⛰       hills      5
🐊       swamp      7
🌋       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾⛰⛰⛰🌾🌾🌾
🌾🌾⛰⛰🌾🌾🌾
🌾🌾⛰🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
['🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌋', '🌋', '🌋', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰'],
['🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '🌾'],
['🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌲', '🌲', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '⛰', '🌾'],
['🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾', '🌾'],
['🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾'],
['🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌾', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🐊', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '⛰', '⛰', '🌋', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌲', '🌲', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌋', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '⛰', '⛰', '⛰', '⛰', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '⛰', '⛰', '🌾', '🐊', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '🌾', '🌾', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🌲', '🌲', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '⛰', '⛰', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊'],
['⛰', '🌋', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '⛰', '⛰', '🌋', '🌋', '🌾', '🌋', '🌋', '⛰', '⛰', '🐊', '🐊', '🐊', '🐊'],
['⛰', '🌋', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🐊', '🐊', '🐊', '🐊'],
['⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾']
]

In [2]:
for line in full_world:
    print("".join(line))

🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌋🌋🌋🌋🌋🌋🌋🌾🌾
🌾🌾🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋⛰⛰⛰🌋🌋⛰⛰
🌾🌾🌾🌾⛰🌋🌋🌋🌲🌲🌲🌲🐊🐊🌲🌲🌲🌲🌲🌾🌾⛰⛰🌋🌋⛰🌾
🌾🌾🌾⛰⛰🌋🌋🌲🌲🌾🌾🐊🐊🐊🐊🌲🌲🌲🌾🌾🌾⛰🌋🌋🌋⛰🌾
🌾⛰⛰⛰🌋🌋⛰⛰🌾🌾🌾🌾🐊🐊🐊🐊🐊🌾🌾🌾🌾🌾⛰🌋⛰🌾🌾
🌾⛰⛰🌋🌋⛰⛰🌾🌾🌾🌾⛰🌋🌋🌋🐊🐊🐊🌾🌾🌾🌾🌾⛰🌾🌾🌾
🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾🌾⛰🌋🌋🌋🐊🐊🐊🌾🌾⛰⛰⛰🌾🌾
🌾🌾🌾⛰⛰⛰🌾🌾🌾🌾🌾🌾⛰⛰🌋🌋🌾🐊🐊🌾🌾⛰⛰⛰🌾🌾🌾
🌾🌾🌾🐊🐊🐊🌾🌾⛰⛰⛰🌋🌋🌋🌋🌾🌾🌾🐊🌾⛰⛰⛰🌾🌾🌾🌾
🌾🌾🐊🐊🐊🐊🐊🌾⛰⛰🌋🌋🌋⛰🌾🌾🌾🌾🌾⛰🌋🌋🌋⛰🌾🌾🌾
🌾🐊🐊🐊🐊🐊🌾🌾⛰🌋🌋⛰🌾🌾🌾🌾🐊🐊🌾🌾⛰🌋🌋⛰🌾🌾🌾
🐊🐊🐊🐊🐊🌾🌾⛰⛰🌋🌋⛰🌾🐊🐊🐊🐊🌾🌾🌾⛰🌋⛰🌾🌾🌾🌾
🌾🐊🐊🐊🐊🌾🌾⛰🌲🌲⛰🌾🌾🌾🌾🐊🐊🐊🐊🌾🌾⛰🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌋🌾🌾🌲🌲🌲🌲⛰⛰⛰⛰🌾🐊🐊🐊🌾🌾⛰🌋⛰🌾🌾🌾
🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌋🌋🌋⛰⛰🌾🐊🌾⛰🌋🌋⛰🌾🌾🌾
🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋🌾🌾🌋🌋🌋🌾🌾🌾🌾🌾
🌾🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋🌋🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊
🌾🌾⛰⛰⛰⛰🌋🌋🌲🌲🌲🌲🌲🌾🌋🌾🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾⛰⛰⛰🌋🌋🌋🌲🌲🌋🌋🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾🌾🌾⛰⛰⛰🌋🌋🌋🌋🌾🌾🌾🌾⛰⛰🌾🌾🐊🐊🐊🐊🐊🐊
🌾⛰⛰🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾⛰⛰🌋🌋⛰⛰🌾🐊🐊🐊🐊🐊
⛰🌋⛰⛰⛰⛰🌾🌾🌾🌾🌾🌋🌋🌋⛰⛰🌋🌋🌾🌋🌋⛰⛰🐊🐊🐊🐊
⛰🌋🌋🌋⛰🌾🌾🌾🌾🌾⛰⛰🌋🌋🌋🌋⛰⛰⛰⛰🌋🌋🌋🐊🐊🐊🐊
⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾⛰⛰⛰🌾🌾🌾


## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [3]:
small_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [4]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]# down, right , up , left

## Costs

We can encode the costs described above in a `Dict`:

In [5]:
COSTS = { '🌾': 1, '🌲': 3, '⛰': 5, '🐊': 7}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (because it has side effects, the printing, before returning anything).

Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
* Do not print raw data structures
* Do not insert unneeded/unrequested spaces
* Use the provided emojis!

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [6]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy
import math

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="successors"></a>
## seccessors



* **state** Tuple[int, int]: the starting location of the the node being checked
* **world** List[List[str]]: the actual context for the navigation problem.

* **returns** 

In [7]:
def successors(state: Tuple[int, int], w:  List[List[str]], m: List[Tuple[int, int]]) -> [Tuple[int, int]]:
    children = []
    world_size_Y = len(w)
    world_size_X = len(w[0])
    for move in m:
        if 0 <= tuple(sum(x) for x in zip(state, move))[0] < world_size_X and 0 <=tuple(sum(x) for x in zip(state, move))[1] < world_size_Y:
            children.append(tuple(sum(x) for x in zip(state, move)))
        else:
            pass # illegal move off of the board
#     print(children)
    
    return children 

<a id="heuristic"></a>
## heuristic

measures the estimated cost of reaching the goal

* **state** Tuple[int, int]: the starting location of the the node being checked
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **world** List[List[str]]: the actual context for the navigation problem.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.

* **returns** Tuple(int, int): The total cost of the heuristic, The cost from start to node n

In [8]:
def heuristic(state: Tuple[int, int], goal: Tuple[int, int], path: List[Tuple[int, int]], world: List[List[str]], costs: Dict[str, int]) -> int:
    f_cost= 0 # total cost
    g_cost= 0 # cost from start to n
    h_cost= 0 # cost from n to goal
    listOfCost = []
    h_cost= math.floor(math.sqrt((goal[0] - state[0])**2 + (goal[1] - state[1])**2))
    for pathStep in path:
        listOfCost.append(costs[world[pathStep[1]][pathStep[0]]])
    g_cost= sum(listOfCost)
    f_cost= g_cost+ h_cost
    return f_cost

In [9]:
assert heuristic((0,0),(3,4), [(0,0)], small_world, COSTS) == 6
assert heuristic((0,6),(3,4), [(0,0)], small_world, COSTS) == 4
assert heuristic((0,6),(3,4), [(0,0), (0,1)], small_world, COSTS) == 5

<a id="a_star_search"></a>
## a_star_search

performs a graph search based on estimated (heristic measure) of cost to reach the goal

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [10]:
def a_star_search( world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    frontier = []
    explored = []
    path = {start:[]}
    frontier.append((start,heuristic(start, goal, path[start], world, costs))) # Tuple(Node,Heuristic)
    while len(frontier) != 0:
        print("path in A*", path)
        print("frontier", frontier)
        current_state = frontier.pop()[0]
        if goal == current_state:
            explored.append(current_state)
            return path[current_state]
        children = successors(current_state, world, moves) # maybe successor should have cost via heuristic. however heuristic requires path which isn't availble at this stage
        for child in children:
            parentList= []
            if child not in explored + frontier: # problem created here because (child, explored, frontier) are not similar for checking this
                parentList.append(current_state)
                path[child]= parentList # create new child parent dict
                frontier.append((child,heuristic(child, goal, path[child], world, costs))) # weigh child first
                frontier.sort(reverse=True)         
        explored.append(current_state)
    return 0 # change to return the real answer

<a id="deltaPath"></a>
## deltaPath
deltaPath measures the displacement of the next move and return a dict{path: arrow}

path List[Tuple[int, int]]: the path from start to goal, in offsets.
returns dict{state: arrow} - The appropriate arrows for the path 

In [11]:
def deltaPath(p):
    trails = {(1,0):'⏩', (-1,0):'⏪', (0,1):'⏫', (0,-1):'⏬'}
    steps={}
    for i in range(len(p)-1):
        steps[p[i]]=trails[(p[i+1][0]-p[i][0],p[i+1][1]-p[i][1])]
    return steps

In [12]:
p1 = ([(0,0),(0,1)])
p2 = ([(0,0),(1,0)])
p3 = ([(0,0),(0,-1)])
p4 = ([(0,0),(-1,0)])
assert  deltaPath(p1) == {(0, 0):'⏫'}
assert  deltaPath(p2) == {(0, 0):'⏩'}
assert  deltaPath(p3) == {(0, 0):'⏬'}
assert  deltaPath(p4) == {(0, 0):'⏪'}

<a id="pretty_print_path"></a>
## pretty_print_path

prints a graph of the word with track while returning a interger measure of the path cost

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** int - The path cost.


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

In [13]:
def pretty_print_path( world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int]) -> int:
    goalPrize = '🎁'
    listOfCost =[]
    for pathStep in path:
        listOfCost.append(costs[world[pathStep[1]][pathStep[0]]])
    for i in path:
        if i == goal:
            world[i[1]][i[0]] = goalPrize
        else:
            world[i[1]][i[0]] = deltaPath(path)[(i[0],i[1])]
    for i in world:
        print("".join(i)) 
    return sum(listOfCost) # replace with the real value!

## Problems

## Problem 1. 

Execute `a_star_search` and `pretty_print_path` for the `small_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [14]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



path in A* {(0, 0): [], (1, 0): [(0, 0)], (0, 1): [(0, 0)], (1, 1): [(1, 0)], (0, 2): [(0, 1)], (1, 2): [(1, 1)], (0, 3): [(0, 2)], (1, 3): [(1, 2)], (0, 4): [(0, 3)], (1, 4): [(1, 3)], (0, 5): [(0, 4)], (1, 5): [(1, 4)], (0, 6): [(0, 5)], (1, 6): [(1, 5)], (2, 0): [(1, 0)], (2, 1): [(2, 0)], (2, 2): [(2, 1)], (2, 3): [(2, 2)], (2, 4): [(2, 3)], (2, 5): [(2, 4)], (2, 6): [(2, 5)], (3, 0): [(2, 0)], (3, 1): [(3, 0)], (3, 2): [(3, 1)], (3, 3): [(3, 2)], (3, 4): [(3, 3)], (3, 5): [(3, 4)], (3, 6): [(3, 5)], (4, 0): [(3, 0)], (4, 1): [(4, 0)], (4, 2): [(4, 1)], (4, 3): [(4, 2)], (4, 4): [(4, 3)], (4, 5): [(4, 4)], (4, 6): [(4, 5)], (5, 0): [(4, 0)], (5, 1): [(5, 0)], (5, 2): [(5, 1)], (5, 3): [(5, 2)], (5, 4): [(5, 3)], (5, 5): [(5, 4)], (5, 6): [(5, 5)], (6, 0): [(5, 0)], (6, 1): [(6, 0)], (6, 2): [(6, 1)], (6, 3): [(6, 2)], (6, 4): [(6, 3)], (6, 5): [(6, 4)], (6, 6): [(6, 5)]}
frontier [((6, 6), 3), ((6, 6), 3), ((6, 6), 3), ((6, 6), 3), ((6, 6), 3), ((6, 6), 3), ((6, 6), 3), ((6, 6), 3)

KeyError: (6, 5)

## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [ ]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

## Comments

(This is the place to leave me comments)

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.